In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pickle

#open pickle files
with open('x_training.pkl', 'rb') as file:
    X_training = pickle.load(file)

with open('y_training.pkl', 'rb') as file:
    y_training = pickle.load(file)

with open('evaluation_x.pkl', 'rb') as file:
    evaluation_x = pickle.load(file)

with open('evaluation_y.pkl', 'rb') as file:
    evaluation_y = pickle.load(file)

with open('Y_pool.pkl', 'rb') as file:
    Y_pool = pickle.load(file)

with open('X_pool', 'rb') as file:
   X_pool = pickle.load(file)

with open('embeddings_unlabeled', 'rb') as file:
    embeddings_unlabeled = pickle.load(file)


In [ ]:
from matplotlib import pyplot as plt
from modAL.models import ActiveLearner
from sklearn.naive_bayes import GaussianNB
from modAL.uncertainty import entropy_sampling


pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)  

# initializing the learner
learner = ActiveLearner(
    estimator=GaussianNB(),
    X_training=X_training, y_training=y_training,
    bootstrap_init=True,
    query_strategy=entropy_sampling,
)

balancedLearner = ActiveLearner(
    estimator=GaussianNB(),
    X_training=X_training, y_training=y_training,
    bootstrap_init=True
)

# query for labels
n_queries = 20
accuracy_scores = [balancedLearner.score(evaluation_x, evaluation_y)]
for idx in range(n_queries):
    # Query for labels from the pool of unlabeled instances
    query_idx, query_instance = balancedLearner.query(X_pool)
    
    # Obtain new labels for the queried instances
    print(embeddings_unlabeled.loc[query_idx]['combined_text'])
    sys.stdout.flush()

    
    # Get the label from the human annotator
    while True:
        user_input = input("Enter the label (0, 1, 2): ")
        try:
            label = int(user_input)
            if label in [0, 1, 2]:
                break
            else:
                print("Label must be one of the following: 0, 1, 2")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")
    
    query_idx = np.array(query_idx, dtype=int)
    
    # Update Y_pool with new labels
    Y_pool[query_idx] = label

    # Supply labels for queried instances
    balancedLearner.teach(X_pool[query_idx], Y_pool[query_idx], bootstrap=True)

    # Update the pool
    X_pool = np.delete(X_pool, query_idx, axis=0)
    Y_pool = np.delete(Y_pool, query_idx, axis=0)

    accuracy_scores.append(balancedLearner.score(evaluation_x, evaluation_y))
    print(accuracy_scores)

plt.figure(figsize=(10, 5))
plt.title('Accuracy of the classifier during the active learning')
plt.plot(range(n_queries+1), accuracy_scores)
plt.scatter(range(n_queries+1), accuracy_scores)
plt.xlabel('number of queries')
plt.ylabel('accuracy')
plt.show()

accuracy_df = pd.DataFrame(accuracy_scores)
accuracy_df.to_csv('accuracies_naivebayes.csv', index=False)